# Tuner TCN

## Imports

In [ ]:
import os
import pandas as pd
import json

import tensorflow as tf
from keras_tuner import Hyperband
from tensorflow import keras
from tensorflow.keras import layers
from tcn import TCN
from tensorflow.keras.layers import Dense, Input, concatenate
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.callbacks import EarlyStopping

## Data prep

In [ ]:
import sys
sys.path.insert(1, '../')

from data_preparation import prepare_x_data, get_Y_labels, reshape_Y, reshape_X

In [ ]:
scaler = StandardScaler()

### X data - features

In [ ]:
# TRAIN data
x_train_visual = prepare_x_data('../Data/LLDs_video_openface/train',',', 5, scaler)
x_train_audio = prepare_x_data('../Data/LLDs_audio_eGeMAPS/train',';', 2, scaler) 

### Y data - labels

In [ ]:
#Y TRAIN labels
y_train = get_Y_labels('../Data/labels_metadata.csv', 60, 164, scaler)
y_train = reshape_Y(y_train,len(x_train_visual),1,1)

## Tuner

In [ ]:
def build_model_visual(hp):
    # Define input layer.
    input = Input(shape=(None, 465))

    # Define hidden layer.
    tcn_v = TCN(nb_filters=hp.Int('v_nb_filters', min_value=1, max_value=465, step=1), 
                kernel_size=hp.Choice('v_kernel_size', [1,2,4,8,16,32,64,128]), 
                dilations=(128,256,512,1024), 
                padding=hp.Choice('v_padding', ['causal', 'same']), 
                nb_stacks=hp.Choice('v_nb_stacks', [1,2]), 
                use_batch_norm=False, 
                use_layer_norm=False,
                use_weight_norm=False, 
                use_skip_connections=True, 
                dropout_rate=hp.Choice('v_dropout_rate', [1e-2, 1e-3, 1e-4]), 
                return_sequences=False, 
                input_shape=(None, 465))(input)

    # Define output layer.
    output = Dense(1, activation='linear')(tcn_v)

    # Define optimizer and show summary.
    model = keras.Model(inputs=[input], outputs=[output])
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])), loss='mse', metrics=['mae'])

    return model

In [ ]:
def build_model_audio(hp):
    # Define input layer.
    input = Input(shape=(None, 23))

    # Define hidden layer.
    tcn_a = TCN(nb_filters=hp.Int('a_nb_filters', min_value=1, max_value=23, step=1), 
                kernel_size=hp.Choice('a_kernel_size', [1,2,4,8,16,32,64,128]), 
                dilations=(128,256,512,1024), 
                padding=hp.Choice('a_padding', ['causal', 'same']), 
                nb_stacks=hp.Choice('a_nb_stacks', [1,2]), 
                use_batch_norm=False, 
                use_layer_norm=False, 
                use_weight_norm=False,
                use_skip_connections=True, 
                dropout_rate=hp.Choice('a_dropout_rate', [1e-2, 1e-3, 1e-4]), 
                return_sequences=False, 
                input_shape=(None, 23))(input)


    # Define output layer.
    output = Dense(1, activation='linear')(tcn_a)

    # Define optimizer and show summary.
    model = keras.Model(inputs=[input], outputs=[output])
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])), loss='mse', metrics=['mae'])

    return model

In [ ]:
 # Define tuner. A new tuner is created for every file.
tuner_v = Hyperband(
    hypermodel=build_model_visual, 
    objective='loss', 
    max_epochs=50, 
    factor=3, 
    project_name='TCN_Tuner_Visual'
)

tuner_a = Hyperband(
    hypermodel=build_model_audio, 
    objective='loss', 
    max_epochs=50, 
    factor=3, 
    project_name='TCN_Tuner_Audio'
)

early_stop = EarlyStopping(monitor='loss', min_delta=0, patience=10, mode='min')

In [ ]:
# Tune the model one file at a time.
#Train visual data tuner
index_v = 0
for train_visual in x_train_visual:
    train_visual = reshape_X(train_visual)

    tuner_v.search(train_visual, y_train[index_v], epochs=10, batch_size=len(train_visual), callbacks=[early_stop]) 
    index_v += 1

In [ ]:
# Tune the model one file at a time.
#Train audio data tuner
index_a = 0
for train_audio in x_train_audio:
    train_audio = reshape_X(train_audio)

    tuner_a.search(train_audio, y_train[index_a], epochs=10, batch_size=len(train_audio), callbacks=[early_stop]) 
    index_a += 1

In [ ]:
#Get df from all tuner searches
def get_hp_df(tunerdir):
    rootdir = tunerdir
    hp_values = []
    #Get learned hyperparameters for every file (trial) and append to a list
    for trial in os.listdir(rootdir):
        pathdir = os.path.join(rootdir, trial)
        filedir = os.path.join(pathdir, 'trial.json')
        
        if os.path.isdir(pathdir): #looking only for trial subfolders
            with open(filedir) as json_file:
                data = json.load(json_file)
                values = data['hyperparameters']['values'] #get learned hyperparameters
                hp_values.append(values) #append to list
        
    #Transform to dataframe
    hp_df = pd.DataFrame(hp_values)

    return hp_df

In [ ]:
# Get average hyperparameter values for both modalities
hpdf_multi_v = get_hp_df('../TCN/TCN_Tuner_Visual')
hpdf_multi_a = get_hp_df('../TCN/TCN_Tuner_Audio')

In [ ]:
print(""" Average hyperparameters for each data modality.
--------FOR VISUAL DATA:""")
hpdf_multi_v.mean()

In [ ]:
print(""" Average hyperparameters for each data modality.
--------FOR AUDIO DATA:""")
hpdf_multi_a.mean()